# 1. Matrix NTRU attack to recover the message


### 1.1 Outline of the attack

1. Generate a key pair for matrix NTRU

2. Generate a message M and encrypted message E. 

3. Use the public key H to construct the lattice L.

4. Apply BKZ to L and obtain Lreduced

5. Construct a matrix from Lreduced using its first upper left block, and  call it Xattack and its Xpattack its inverse mod p

6. Use Xattack and Xpattack to decrypt E and check it equals to M.

# 2. Start of code

In [ ]:
import numpy as np

load("matrix_ntru_system.sage")

def reduceL(H,n,p,q):
    AB = np.concatenate((identity_matrix(n), p.inverse_mod(q) * H), axis=1)
    CD = np.concatenate((0*identity_matrix(n), q*identity_matrix(n)), axis=1)
    L = matrix(ZZ,np.concatenate((AB,CD),axis=0))
    LBKZ = L.BKZ()
    return LBKZ[0:n,0:n]


def attack(n,q, verbose = False):

    try:
        # n,q = 4,32
        # STEP 1
        setParameters([n,q]) # parameters n and q.
        X,Y,Xp,Xq,H = keygen()
        del X,Y,Xp,Xq # to ensure we are not using it to decrypt
        
        # STEP 2
        M = randomMessage()
        E = encrypt(M,H)
        
        # STEP 3, 4 and 5
        Xattack = reduceL(H,n,p,q)
        Xpattack = Matrix(Rp,Xattack).inverse()
        
        # STEP 5
        Mattack = decrypt(E,Xattack,Xpattack)
        
        # STEP 6
        Result = Mattack == M
        
        
        # Print if Verbose
        if verbose:
            # sanity check 
            X,Y,Xp,Xq,H = keygen()
            print("decryption with attack key:", Mattack == M)
            print("decryption with wrong key:", decrypt(E,X,Xp) == M)
            
            # print message and message recovered with the attacked key
            print("M \n{} \n Mattack \n{}".format(M,Mattack))
        return Result
    except:
        return -1

In [ ]:
# run the attack once
attack(10,4096,True)

### 2.1 Simulation study for the attack performance for several values of n and fixed q

In [ ]:
nvalues = [5,10,20,30,40,50,60,70,80,90,100,110]
q = 256
rep = 100
result = matrix(ZZ,len(nvalues),rep)

for i in range(len(nvalues)):
    print('running for n = ' + str(nvalues[i]))
    setParameters([nvalues[i],q]) # parameters n and q.
    for j in range(rep):
        result[i,j] = attack(nvalues[i],q)
#result

import pickle
fname = "matrix_ntru_atack_q_{}_rep_{}.pkl".format(q,rep)
with open(fname,"wb") as f:
    pickle.dump(result,f)

### 2.2 plots for the article

In [ ]:
import pickle
with open("matrix_ntru_atack_q_{}_rep_{}.pkl".format(4096,100),"rb") as f:
     res = pickle.load(f)
import matplotlib.pyplot as plt
from sage.matrix.constructor import Matrix

# Calculate the percentage of 1's in each row
percentages = [sum([i == 1 for i in row]) / rep * 1.0 for row in result]

# Plot the percentage values against the corresponding values of n
plt.plot(nvalues, percentages, marker='o')
plt.xlabel('n')
plt.ylabel('Percentage of Sucess')
plt.title('q = ' + str(q))
plt.grid(True)
plt.show()